Many examples are extensions of https://github.com/PacktPublishing/Interactive-Dashboards-and-Data-Apps-with-Plotly-and-Dash
which are from [Interactive Dashboards and Data Apps with Plotly and Dash](https://www.packtpub.com/product/interactive-dashboards-and-data-apps-with-plotly-and-dash/9781800568914?utm_source=github&utm_medium=repository&utm_campaign=9781800568914)

In [1]:
import intersystems_iris.dbapi._DBAPI as dbapi
from dotenv import load_dotenv
load_dotenv()
import os

# See example from https://github.com/intersystems-community/intersystems-irispython
# Is some issues with the other driver, so use this one.

password = os.getenv("SQL_PASSWORD")
user = os.getenv("SQL_USERNAME")
host = os.getenv("SQL_SERVER")
namespace = os.getenv("SQL_NAMESPACE")
port = os.getenv("SQL_PORT")
if isinstance(port, str):
    port = int(port)

config = {
    "hostname": host,
    "port": port,
    "namespace": namespace,
    "username": user,
    "password": password,
}

try:
    conn = dbapi.connect(**config)
    print("Connection successful!")
except Exception as e:
    print(f"Failed to connect: {e}")

# create a cursor
cursor = conn.cursor()
cursor


Connection successful!


In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "notebook_connected"


sql = """
      select ResourceType, count(*) Total from HSFHIR_X0001_R.Rsrc group by ResourceType
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
dfResource = pd.DataFrame(data, columns=column_names)

figResource = go.Figure(data=[go.Table(
    header=dict(values=list(dfResource.columns),
                align='left'),
    cells=dict(values=[dfResource.ResourceType, dfResource.Total],
               align='left'))
])

figResource.show()


In [3]:
sql = """
      select dr.ID1,CDR.GetJSON(ResourceString,'basedOn'),$PIECE(CDR.GetFHIRPathOne(ResourceString,'basedOn[0].reference','DiagnosticReport'),'/',2) ServiceRequest,sr.requester_IdentifierValue from HSFHIR_X0001_S.DiagnosticReport dr
  join HSFHIR_X0001_R.Rsrc resource on resource.ResourceType = 'DiagnosticReport'
          and resource.Key = dr.Key
  left outer join HSFHIR_X0001_S.ServiceRequest sr on CDR.GetFHIRPathOne(ResourceString,'basedOn[0].reference','DiagnosticReport')=sr.key
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)

df1 = df.groupby(['requester_IdentifierValue']).size().reset_index(name='count')

df1

,requester_IdentifierValue,count
0,IRL,1
1,R0A,2
2,REP,2
3,RWJ,1


In [4]:
sql = """
      select
          dr.ID1
           ,sr.requester_IdentifierValue ODS
           ,org.phonetic name
           ,org.partof_IdentifierValue ICS
           ,parent.phonetic ICSName
      from HSFHIR_X0001_S.DiagnosticReport dr                                             join HSFHIR_X0001_S_DiagnosticReport.basedOn drb on drb.Key = dr.Key               left outer join HSFHIR_X0001_S.ServiceRequest sr on drb.value_Reference=sr.key
      left outer join HSFHIR_X0001_S.Organization org on org.key = sr.requester_RelativeRef                                                             left outer join HSFHIR_X0001_S.Organization parent on parent.key = org.partof_Reference
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)

dfOrg = df.groupby(['ODS','name']).size().reset_index(name='Total')
dfICS = df.groupby(['ICS','ICSName']).size().reset_index(name='Total')

figOrg = go.Figure(data=[go.Table(
    header=dict(values=list(dfOrg.columns),
                align='left'),
    cells=dict(values=[dfOrg.ODS, dfOrg.name,  dfOrg.Total],
               align='left'))
])
figICS = go.Figure(data=[go.Table(
    header=dict(values=list(dfICS.columns),
                align='left'),
    cells=dict(values=[ dfICS.ICS, dfICS.ICSName, dfICS.Total],
               align='left'))
])
figOrg.show()
figICS.show()

Order Number Investigations

In [5]:
sql = """
      select
          dr.ID1
           , src.value_Value OrderCode
           ,sri.value_Value OrderPlacerNumber
           ,sr.authored_StartRaw OrderDate
           ,sr.requester_IdentifierValue OrderOrganisationODS
           ,org.phonetic OrderOrganisationName
      from HSFHIR_X0001_S.DiagnosticReport dr                                                                                                 join HSFHIR_X0001_S_DiagnosticReport.basedOn drb on drb.Key = dr.Key
                                                                                                                                              left outer join HSFHIR_X0001_S.ServiceRequest sr on drb.value_Reference=sr.key
                                                                                                                                              join HSFHIR_X0001_S_ServiceRequest.code src on src.Key = sr.Key
                                                                                                                                              left outer join HSFHIR_X0001_S_ServiceRequest.identifier
          sri on sr.key=sri.key and sri.value_TypeCodingCode='PLAC'
                                                                                                                                              left outer join HSFHIR_X0001_S.Organization org on org.key = sr.requester_RelativeRef                                                                                           left outer join HSFHIR_X0001_S.Organization parent on parent.key = org.partof_Reference
                                                                                                                                              left outer join HSFHIR_X0001_S_Patient.identifier pat on pat.key=dr.patient_Reference and pat.value_TypeCodingCode = 'MR'
                                                                                                                                              left outer join HSFHIR_X0001_S_Patient.identifier pat2 on pat2.key=dr.patient_Reference and pat2.value_TypeCodingCode = 'NH'
      where sr.requester_IdentifierValue <> 'R0A'
      order by sr.requester_IdentifierValue
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)

df

,ID1,OrderCode,OrderPlacerNumber,OrderDate,OrderOrganisationODS,OrderOrganisationName
0,9,R112.1,None,2025-10-02,IRL,None
1,2,R125.1,1000152868,None,REP,LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
2,2,R125.1,1000152868,None,REP,LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
3,2,R125.1,1000152868,None,REP,LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
4,2,R125.1,1000152868,None,REP,LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
5,2,R125.1,1000152868,None,REP,LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
6,2,R125.1,1000152868,None,REP,LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
7,2,R125.1,1000152868,None,REP,LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
8,2,R125.1,1000152868,None,REP,LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
9,2,R125.1,1000152868,None,REP,LIVERPOOL WOMEN'S NHS FOUNDATION TRUST


List of tests and Test Directory Codes

In [6]:
sql = """
      select test.value_Value iGeneCode
           ,test.value_Text iGeneDisplay
           ,directory.value_Value TestDirectoryCode
           ,directory.value_Text TestDirectoryDisplay
      from HSFHIR_X0001_S_DiagnosticReport.code test
               join HSFHIR_X0001_S_DiagnosticReport.code directory on test.Key = directory.Key and directory.value_System = 'https://fhir.nhs.uk/CodeSystem/England-GenomicTestDirectory'
      where test.value_System = 'https://fhir.nwgenomics.nhs.uk/CodeSystem/IGEAP'
      group by test.value_Value, directory.value_Value
      order by test.value_Value
"""

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)

df

,iGeneCode,iGeneDisplay,TestDirectoryCode,TestDirectoryDisplay
0,CCP19V1_R112,PANEL: R112.1 - Factor II deficiency v1.0,R112.1,Factor II deficiency (Single gene sequencing >...
1,M80_AML_GX,PANEL: M80_AML_GX,M80.2,"Acute Myeloid Leukaemia, Multi-target NGS pane..."
2,PCR-FLT3-ITD,TEST: PCR - FLT3 ITD mutation screen,M80.18,"Acute Myeloid Leukaemia, FLT3 ITD"
3,PCR-FLT3-ITD,TEST: PCR - FLT3 ITD mutation screen,M80.21,"Acute Myeloid Leukaemia, FLT3 TKD hotspot"
4,R125_CARDIOLOGY,"PANEL: R125.1, R125.2 - Thoracic aortic aneury...",R125.1,Thoracic aortic aneurysm or dissection (WES or...
5,SANGER-FAMILY-FOLLOW-UP-RARE,TEST: Sanger Sequencing - Family follow-up tes...,R375.1,Family follow-up testing to aid variant interp...


In [7]:
import dash
from dash import dcc, html, Input, Output, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go

# Initialize the Dash app (no longer need JupyterDash)
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

@app.callback(
    Output('fhirresources', 'figure')
)
def fhirresources():

    figResource = go.Figure(data=[go.Table(
        header=dict(values=list(dfResource.columns),
                    align='left'),
        cells=dict(values=[dfResource.ResourceType, dfResource.Total],
                   align='left'))
    ])
    return figResource

figResource = fhirresources

app.layout = html.Div([
    html.H1("NW Genomics HIE SQL Dashboard"),

    dbc.Row([
        dbc.Col([
            html.H3("NHS Trust Report Totals"),
            dash_table.DataTable(data = dfOrg.to_dict('records'),
                             columns =  [{"name": i, "id": i} for i in dfOrg.columns],
                            style_cell=dict(textAlign='left'),
                            style_header=dict(backgroundColor="paleturquoise"),
                            style_data=dict(backgroundColor="lavender")),
        ]),
        dbc.Col([
            html.H3("ICS Report Totals"),
            dash_table.DataTable(data = dfICS.to_dict('records'),
                             columns =  [{"name": i, "id": i} for i in dfICS.columns],
                             style_cell=dict(textAlign='left'),
                             style_header=dict(backgroundColor="paleturquoise"),
                             style_data=dict(backgroundColor="lavender"))
        ])
    ]),
    dbc.Row([
        html.H3("FHIR Resources"),
        #  dcc.Graph(id='fhirresources', figure=figResource),
        dash_table.DataTable(data = dfResource.to_dict('records'),
                             columns = [{"name": i, "id": i} for i in dfResource.columns],
                             style_cell=dict(textAlign='left'),
                             style_header=dict(backgroundColor="paleturquoise"),
                             style_data=dict(backgroundColor="lavender"))
    ])
])

if __name__ == '__main__':
    # Use app.run with jupyter_mode instead of JupyterDash's run_server
    app.run(jupyter_mode='inline', port=8053)